In [1]:
# Dependencies
import json
import pandas as pd
import numpy as np

# Import and read in CSV
youtube_df = pd.read_csv('./Data/USvideos.csv')

# Import Json and create category csv to read in excel if needed
# cat_id_df = pd.read_json('./Data/US_category_id.json')
# cat_id_df.to_csv('./Data/Category_alt.csv')

# Find out what categories are being used
youtube_df.category_id.value_counts()

24    9964
10    6472
26    4146
23    3457
22    3210
25    2487
28    2401
1     2345
17    2174
27    1656
15     920
20     817
19     402
2      384
29      57
43      57
Name: category_id, dtype: int64

In [2]:
# Create a dictionary for the categories used
# Reference the 'Category_alt.csv'
# Create a dict
categories = {1:'Film & Animation',
              2:'Autos & Vehicles',
              10:'Music',
              15:'Pets & Animals',
              17:'Sports',
              19:'Travel & Events',
              20:'Gaming',
              22:'People & Blogs',
              23:'Comedy',
              24:'Entertainment',
              25:'News & Politics',
              26:'Howto & Style',
              27:'Education',
              28:'Science & Technology',
              29:'Nonprofits & Activism',
              43:'Shows' 
             }

In [3]:
# Create a duplicate of 'category_id' to apply the dictionary to
youtube_df['category_type'] = youtube_df.loc[:,'category_id']
# Apply the dictionary
youtube_df.category_type = [categories[item] for item in youtube_df.category_type]

In [4]:
# Check to see if the dictornary was applied properly
youtube_df.category_type.value_counts()

Entertainment            9964
Music                    6472
Howto & Style            4146
Comedy                   3457
People & Blogs           3210
News & Politics          2487
Science & Technology     2401
Film & Animation         2345
Sports                   2174
Education                1656
Pets & Animals            920
Gaming                    817
Travel & Events           402
Autos & Vehicles          384
Nonprofits & Activism      57
Shows                      57
Name: category_type, dtype: int64

In [5]:
youtube_df = youtube_df.drop(['category_id', 'video_id', 'thumbnail_link'], axis=1)

In [6]:
youtube_df

,trending_date,title,channel_title,publish_time,tags,views,likes,dislikes,comment_count,comments_disabled,ratings_disabled,video_error_or_removed,description,category_type
0,17.14.11,WE WANT TO TALK ABOUT OUR MARRIAGE,CaseyNeistat,2017-11-13T17:13:01.000Z,SHANtell martin,748374,57527,2966,15954,False,False,False,SHANTELL'S CHANNEL - https://www.youtube.com/s...,People & Blogs
1,17.14.11,The Trump Presidency: Last Week Tonight with J...,LastWeekTonight,2017-11-13T07:30:00.000Z,"last week tonight trump presidency|""last week ...",2418783,97185,6146,12703,False,False,False,"One year after the presidential election, John...",Entertainment
2,17.14.11,"Racist Superman | Rudy Mancuso, King Bach & Le...",Rudy Mancuso,2017-11-12T19:05:24.000Z,"racist superman|""rudy""|""mancuso""|""king""|""bach""...",3191434,146033,5339,8181,False,False,False,WATCH MY PREVIOUS VIDEO ▶ \n\nSUBSCRIBE ► http...,Comedy
3,17.14.11,Nickelback Lyrics: Real or Fake?,Good Mythical Morning,2017-11-13T11:00:04.000Z,"rhett and link|""gmm""|""good mythical morning""|""...",343168,10172,666,2146,False,False,False,Today we find out if Link is a Nickelback amat...,Entertainment
4,17.14.11,I Dare You: GOING BALD!?,nigahiga,2017-11-12T18:01:41.000Z,"ryan|""higa""|""higatv""|""nigahiga""|""i dare you""|""...",2095731,132235,1989,17518,False,False,False,I know it's been a while since we did this sho...,Entertainment
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40944,18.14.06,The Cat Who Caught the Laser,AaronsAnimals,2018-05-18T13:00:04.000Z,"aarons animals|""aarons""|""animals""|""cat""|""cats""...",1685609,38160,1385,2657,False,False,False,The Cat Who Caught the Laser - Aaron's Animals,Pets & Animals
40945,18.14.06,True Facts : Ant Mutualism,zefrank1,2018-05-18T01:00:06.000Z,[none],1064798,60008,382,3936,False,False,False,NaN,People & Blogs
40946,18.14.06,I GAVE SAFIYA NYGAARD A PERFECT HAIR MAKEOVER ...,Brad Mondo,2018-05-18T17:34:22.000Z,I gave safiya nygaard a perfect hair makeover ...,1066451,48068,1032,3992,False,False,False,I had so much fun transforming Safiyas hair in...,Entertainment
40947,18.14.06,How Black Panther Should Have Ended,How It Should Have Ended,2018-05-17T17:00:04.000Z,"Black Panther|""HISHE""|""Marvel""|""Infinity War""|...",5660813,192957,2846,13088,False,False,False,How Black Panther Should Have EndedWatch More ...,Film & Animation


In [7]:
# youtube_df.to_csv('./Data/USvideos_c1.csv')